In [59]:
import pandas as pd
from pathlib import Path
import json

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

pd.options.display.max_columns = None

In [3]:
# path of the folder with .parquet
folder_path = Path("C:/Users/AULA04/Desktop/Codes/data/box_builder_dataset/feature_frame.csv")

In [4]:
feature_frame = pd.read_csv(folder_path)
feature_frame
#copy, if necesary reload the dataset
feature_frame_org=feature_frame[:]

In [20]:
feature_frame = feature_frame_org

In [21]:
feature_frame=feature_frame[feature_frame["user_order_seq"]>=5].reset_index()

In [22]:
feature_frame[["days_since_purchase_variant_id","avg_days_to_buy_variant_id","std_days_to_buy_variant_id","days_since_purchase_product_type","avg_days_to_buy_product_type","std_days_to_buy_product_type"]].corr()

,days_since_purchase_variant_id,avg_days_to_buy_variant_id,std_days_to_buy_variant_id,days_since_purchase_product_type,avg_days_to_buy_product_type,std_days_to_buy_product_type
days_since_purchase_variant_id,1.000000,0.040183,0.047066,0.178246,0.010523,0.013018
avg_days_to_buy_variant_id,0.040183,1.000000,0.599918,0.008753,0.293421,0.251989
std_days_to_buy_variant_id,0.047066,0.599918,1.000000,0.008725,0.224434,0.255415
days_since_purchase_product_type,0.178246,0.008753,0.008725,1.000000,0.050448,0.053402
avg_days_to_buy_product_type,0.010523,0.293421,0.224434,0.050448,1.000000,0.838906
std_days_to_buy_product_type,0.013018,0.251989,0.255415,0.053402,0.838906,1.000000


In [9]:
(feature_frame["count_adults"] == 2).value_counts()

count_adults
True     447206
False     29396
Name: count, dtype: int64

In [19]:
feature_frame[["count_adults",	"count_children","count_babies","count_pets","people_ex_baby"]]

,count_adults,count_children,count_babies,count_pets,people_ex_baby
6,2.0,0.0,0.0,0.0,2.0
7,2.0,0.0,0.0,0.0,2.0
12,2.0,0.0,0.0,0.0,2.0
13,2.0,0.0,0.0,0.0,2.0
17,2.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...
2880512,2.0,0.0,0.0,1.0,2.0
2880518,2.0,0.0,0.0,0.0,2.0
2880530,2.0,0.0,0.0,0.0,2.0
2880535,2.0,0.0,0.0,0.0,2.0


# Columns
* variant id                            : -> Not include, inside of the model will be the caracteristics 
* product_type                          : -> One Hot encoding
* order_id                              :
* user_id                               :
* X created_at                          : are the same row, have the hour, it's innecesary -> *Delete*
* X order_date                          : There isn't any related to the time, I think it's innecesary here -> *Delete*
* ok user_order_seq                     :
* ok outcome                            :
* ok ordered_before                     :
* ok abandoned_before                   :
* ok active_snoozed                     :
* ok set_as_regular                     :
* normalised_price                      :
* discount_pct                          :
* vendor                                :
* ok global_popularity                  :
* count_adults                          :|
* count_children                        :|
* count_babies                          :|-Datos imputados
* count_pets                            :|
* people_ex_baby                        :|
* ok days_since_purchase_variant_id     :
* ok avg_days_to_buy_variant_id         :
* ok std_days_to_buy_variant_id         :
* ok days_since_purchase_product_type   :   
* ok avg_days_to_buy_product_type       :
* ok std_days_to_buy_product_type       :


### Product type

In [ ]:
#Product type: need to be categorical encoding
Product_type_one_hot = pd.get_dummies(feature_frame["product_type"])
pd.concat([feature_frame, Product_type_one_hot],axis=1)
feature_frame = feature_frame.drop("product_type",axis=1)
feature_frame = feature_frame.drop("index",axis=1)
feature_frame

### Delete: 
* user_id
* variant_id
* order_date
* created_at

In [ ]:
feature_frame = feature_frame.drop("variant_id",axis=1)
feature_frame = feature_frame.drop("user_id",axis=1)
feature_frame = feature_frame.drop("order_date",axis=1)
feature_frame = feature_frame.drop("created_at",axis=1)
feature_frame


,order_id,user_order_seq,outcome,ordered_before,abandoned_before,active_snoozed,set_as_regular,normalised_price,discount_pct,vendor,global_popularity,count_adults,count_children,count_babies,count_pets,people_ex_baby,days_since_purchase_variant_id,avg_days_to_buy_variant_id,std_days_to_buy_variant_id,days_since_purchase_product_type,avg_days_to_buy_product_type,std_days_to_buy_product_type
0,2808548917380,5,0.0,1.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.038462,2.0,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.276180
1,2808549900420,13,0.0,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.038462,2.0,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.276180
2,2809061539972,14,0.0,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.026316,2.0,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.276180
3,2809064751236,15,0.0,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.026316,2.0,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.276180
4,2809189335172,9,0.0,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.026316,2.0,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.276180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476597,3642578272388,5,0.0,0.0,0.0,0.0,0.0,0.417186,0.114360,colief,0.000000,2.0,0.0,0.0,1.0,2.0,33.0,34.0,27.693045,30.0,34.0,27.451392
476598,3642708230276,5,0.0,0.0,0.0,0.0,0.0,0.417186,0.114360,colief,0.000000,2.0,0.0,0.0,0.0,2.0,33.0,34.0,27.693045,30.0,34.0,27.451392
476599,3643130314884,6,0.0,0.0,0.0,0.0,0.0,0.417186,0.114360,colief,0.000000,2.0,0.0,0.0,0.0,2.0,33.0,34.0,27.693045,30.0,34.0,27.451392
476600,3643189461124,9,0.0,0.0,0.0,0.0,0.0,0.417186,0.114360,colief,0.000000,2.0,0.0,0.0,0.0,2.0,33.0,34.0,27.693045,30.0,34.0,27.451392


### order_ID

In [ ]:
#Count encoding
counts = feature_frame.order_id.value_counts()
feature_frame['order_id_2'] = feature_frame.order_id.map(counts)



In [61]:
scaler = MinMaxScaler()
dsf = scaler.fit_transform(feature_frame['order_id_2'])

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [ ]:
feature_frame['order_id_2'].reset_index()